In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    #input_index = np.array(range(0, length, 24))
    input_index = np.array(range(inputs_day.shape[0]))
    #label_index = input_index + 24* (7 + predict_day)
    label_index = input_index + (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    o_ = np.mean(label_day[:,predict_day,:])
    p_ = np.mean(pred_day[:,predict_day,:])
    
    oi = label_day[:,predict_day,:]
    pi = pred_day[:,predict_day,:]
    
    high = ((oi-o_)*(pi-p_)).sum()
    low = np.sqrt( ( (oi-o_)**2 ).sum() )
    low = low * np.sqrt( ( (pi-p_)**2 ).sum() )
    
    R = high/low
    RS = (R)**2
    
    return nse3, pbias3, pred_day, label_day, mae, rmse, RS, R

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e) 


1 Physical GPUs, 1 Logical GPUs


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'geum_pull'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 15)

In [12]:
run_num = range(len(folder))
#if file_parameters['watershed'] == 'nak':
#    run_num = range(len(folder))
#else:
#    run_num = [0]

real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

ddday = 31
mmmonth = 12

#length = 3

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                               colum_idx[idx], interpolation=interpolation_option[idx],
                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/geum/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['현도_2016.xlsx', '현도_2017.xlsx', '현도_2018.xlsx'], ['대청호_2016.xlsx', '대청호_2017.xlsx', '대청호_2018.xlsx'], ['장계_2016.xlsx', '장계_2017.xlsx', '장계_2018.xlsx']]
Epoch 1/2000
1/1 [==============================] - 0s 195ms/step - gen_loss: 131.4533 - disc_loss: 0.7276 - rmse: 1.2691 - val_loss: 0.9095
Epoch 2/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 108.1552 - disc_loss: 0.5348 - rmse: 1.0530 - val_loss: 1.0521
Epoch 3/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 89.5407 - disc_loss: 0.5034 - rmse: 0.9467 - val_loss: 0.9556
Epoch 4/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 77.2321 - disc_loss: 0.4207 - rmse: 0.8659 - val_loss: 0.6534
Epoch 5/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 55.7217 - disc_loss: 0.3791 - rmse: 0.6482 - val_loss: 0.6502
Epoch 6/2000
1/1 [==============

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.5155 - disc_loss: 0.3578 - rmse: 0.2815 - val_loss: 0.2956
Epoch 58/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1294 - disc_loss: 0.3515 - rmse: 0.2602 - val_loss: 0.2614
Epoch 59/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 16.2802 - disc_loss: 0.3628 - rmse: 0.5052 - val_loss: 0.2735
Epoch 60/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 21.5287 - disc_loss: 0.3318 - rmse: 0.5422 - val_loss: 0.3121
Epoch 61/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0179 - disc_loss: 0.3640 - rmse: 0.2801 - val_loss: 0.4745
Epoch 62/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.4196 - disc_loss: 0.3453 - rmse: 0.5148 - val_loss: 0.5131
Epoch 63/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 20.9309 - disc_loss: 0.3336 - rmse: 0.5618 - val_loss: 0.3131
Epoch 64/2000
1/1 [==================

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5204 - disc_loss: 0.2822 - rmse: 0.2383 - val_loss: 0.5483
Epoch 116/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.3236 - disc_loss: 0.2976 - rmse: 0.2720 - val_loss: 0.2486
Epoch 117/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.4157 - disc_loss: 0.2811 - rmse: 0.4283 - val_loss: 0.2781
Epoch 118/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.7376 - disc_loss: 0.2951 - rmse: 0.3423 - val_loss: 0.2379
Epoch 119/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 22.0856 - disc_loss: 0.2867 - rmse: 0.5884 - val_loss: 0.2468
Epoch 120/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 15.5445 - disc_loss: 0.2683 - rmse: 0.5877 - val_loss: 0.2276
Epoch 121/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7518 - disc_loss: 0.2742 - rmse: 0.2567 - val_loss: 0.2328
Epoch 122/2000
1/1 [============

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7780 - disc_loss: 0.2616 - rmse: 0.2324 - val_loss: 0.3517
Epoch 174/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 20.7059 - disc_loss: 0.2546 - rmse: 0.5590 - val_loss: 0.2347
Epoch 175/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4000 - disc_loss: 0.2532 - rmse: 0.2239 - val_loss: 0.5542
Epoch 176/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7554 - disc_loss: 0.2575 - rmse: 0.3146 - val_loss: 0.2226
Epoch 177/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 10.6335 - disc_loss: 0.2642 - rmse: 0.4346 - val_loss: 0.2246
Epoch 178/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.9489 - disc_loss: 0.2727 - rmse: 0.2927 - val_loss: 0.5421
Epoch 179/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8689 - disc_loss: 0.2585 - rmse: 0.2460 - val_loss: 0.2208
Epoch 180/2000
1/1 [=============

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2859 - disc_loss: 0.2446 - rmse: 0.1914 - val_loss: 0.4231
Epoch 232/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5767 - disc_loss: 0.2634 - rmse: 0.2139 - val_loss: 0.1979
Epoch 233/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0040 - disc_loss: 0.2505 - rmse: 0.2195 - val_loss: 0.4078
Epoch 234/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0036 - disc_loss: 0.2443 - rmse: 0.3133 - val_loss: 0.2493
Epoch 235/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1936 - disc_loss: 0.2518 - rmse: 0.2115 - val_loss: 0.2535
Epoch 236/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0429 - disc_loss: 0.2544 - rmse: 0.2029 - val_loss: 0.2040
Epoch 237/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.6951 - disc_loss: 0.2418 - rmse: 0.4210 - val_loss: 0.3864
Epoch 238/2000
1/1 [===============

...

1/1 [==============================] - 0s 21ms/step - gen_loss: 2.3539 - disc_loss: 0.1526 - rmse: 0.2057 - val_loss: 0.1831
Epoch 1798/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4050 - disc_loss: 0.1510 - rmse: 0.1528 - val_loss: 0.4069
Epoch 1799/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.1818 - disc_loss: 0.1555 - rmse: 0.2057 - val_loss: 0.2216
Epoch 1800/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.0793 - disc_loss: 0.1612 - rmse: 0.1799 - val_loss: 0.3926
Epoch 1801/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.9327 - disc_loss: 0.1784 - rmse: 0.2644 - val_loss: 0.1454
Epoch 1802/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 1.7307 - disc_loss: 0.1436 - rmse: 0.1368 - val_loss: 0.1587
Epoch 1803/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.0519 - disc_loss: 0.1526 - rmse: 0.1570 - val_loss: 0.1565
Epoch 1804/2000
1/1 [=======

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6626 - disc_loss: 0.1482 - rmse: 0.1806 - val_loss: 0.1707
Epoch 1856/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1644 - disc_loss: 0.1479 - rmse: 0.1959 - val_loss: 0.1794
Epoch 1857/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4142 - disc_loss: 0.1533 - rmse: 0.1804 - val_loss: 0.1801
Epoch 1858/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.7395 - disc_loss: 0.1481 - rmse: 0.3044 - val_loss: 0.5243
Epoch 1859/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 6.0609 - disc_loss: 0.1488 - rmse: 0.2057 - val_loss: 0.4396
Epoch 1860/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.1233 - disc_loss: 0.1551 - rmse: 0.1802 - val_loss: 0.1989
Epoch 1861/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.0038 - disc_loss: 0.1493 - rmse: 0.2678 - val_loss: 0.1697
Epoch 1862/2000
1/1 [=======

1/1 [==============================] - 0s 146ms/step - gen_loss: 4.1718 - disc_loss: 0.1601 - rmse: 0.2034 - val_loss: 0.1534
Epoch 1914/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.0583 - disc_loss: 0.1467 - rmse: 0.1533 - val_loss: 0.1935
Epoch 1915/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.9919 - disc_loss: 0.1500 - rmse: 0.2509 - val_loss: 0.1708
Epoch 1916/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 1.8275 - disc_loss: 0.1522 - rmse: 0.1467 - val_loss: 0.2354
Epoch 1917/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9804 - disc_loss: 0.1466 - rmse: 0.1864 - val_loss: 0.1911
Epoch 1918/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.4560 - disc_loss: 0.1479 - rmse: 0.1933 - val_loss: 0.2067
Epoch 1919/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.4522 - disc_loss: 0.1604 - rmse: 0.1972 - val_loss: 0.1921
Epoch 1920/2000
1/1 [======

1/1 [==============================] - 0s 20ms/step - gen_loss: 7.2521 - disc_loss: 0.1471 - rmse: 0.2203 - val_loss: 0.1733
Epoch 1972/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.2975 - disc_loss: 0.1428 - rmse: 0.1656 - val_loss: 0.2496
Epoch 1973/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.3782 - disc_loss: 0.1474 - rmse: 0.1769 - val_loss: 0.1688
Epoch 1974/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.6695 - disc_loss: 0.1500 - rmse: 0.1775 - val_loss: 0.2057
Epoch 1975/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.2323 - disc_loss: 0.1453 - rmse: 0.1537 - val_loss: 0.2132
Epoch 1976/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3607 - disc_loss: 0.1425 - rmse: 0.1769 - val_loss: 0.1558
Epoch 1977/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8983 - disc_loss: 0.1384 - rmse: 0.1755 - val_loss: 0.1642
Epoch 1978/2000
1/1 [========

row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
interpol flag :  [True, True]
folder :  data/geum/기타수질오염원/
colum_idx :  :,5:
file_names[idx] :  [['1_2016.xlsx', '1_2017.xlsx', '1_2018.xlsx'], ['2_2016.xlsx', '2_2017.xlsx', '2_2018.xlsx'], ['3_2016.xlsx', '3_2017.xlsx', '3_2018.xlsx'], ['4_2016.xlsx', '4_2017.xlsx', '4_2018.xlsx'], ['5_2016.xlsx', '5_2017.xlsx', '5_2018.xlsx'], ['6_2016.xlsx', '6_2017.xlsx', '6_2018.xlsx']]
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in


In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:

#pd.DataFrame(gan).tail
#real_df_all.tail



In [16]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  2
out_num_features :  1


In [92]:
#ctrl f -- 타겟

print(real_df_all.shape)
rnn_target_column = 'toc'
print(rnn_target_column)



(26304, 704)
toc


In [93]:

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

train_df, test_df, val_df = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, val=1, test=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (18412, 704) val_df.shape :  (5261, 704) test_df.shape: (2631, 704)


In [94]:
#fake_df.shape[0]/8760

In [95]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)


label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [96]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

In [97]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/geum/models/geum_pull/toc/


In [98]:
#rnn_epochs, rnn_in_setps
__RNN_TRAINING__ = True
rnn_continue_train = False
rnn_epochs = 15
rnn_steps_per_epoch = 5

## 모델 학습

In [99]:
gru_model = model_gru(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, 
    epochs=rnn_epochs, steps_per_epoch = rnn_steps_per_epoch,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=rnn_continue_train)

Epoch 1/15
5/5 [==============================] - 85s 17s/step - loss: 0.8367 - mean_absolute_error: 0.7305 - nse: -0.6187 - val_loss: 0.1623 - val_mean_absolute_error: 0.3211 - val_nse: -0.2629
Epoch 2/15
5/5 [==============================] - 78s 16s/step - loss: 0.4318 - mean_absolute_error: 0.5171 - nse: 0.1530 - val_loss: 0.1841 - val_mean_absolute_error: 0.3440 - val_nse: -0.4325
Epoch 3/15
5/5 [==============================] - 77s 15s/step - loss: 0.3027 - mean_absolute_error: 0.4324 - nse: 0.3699 - val_loss: 0.2201 - val_mean_absolute_error: 0.3754 - val_nse: -0.7112
Epoch 4/15
5/5 [==============================] - 78s 16s/step - loss: 0.2402 - mean_absolute_error: 0.3825 - nse: 0.4837 - val_loss: 0.1853 - val_mean_absolute_error: 0.3388 - val_nse: -0.4453
Epoch 5/15
5/5 [==============================] - 77s 15s/step - loss: 0.2022 - mean_absolute_error: 0.3504 - nse: 0.6075 - val_loss: 0.1684 - val_mean_absolute_error: 0.3251 - val_nse: -0.3114
Epoch 6/15
5/5 [=============

In [110]:
rnn_predict_day = 2
rnn_predict_day

2

In [111]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)
    
print(model_path)
print('rnn_predict_day : ', rnn_predict_day + 1)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column , " ", out_features[0])
print("length : ",real_df_all.shape[0])
print(teg_check)

print('all_nse : ', val_nse['GRU'])
print('all_pbias : ', val_pbias['GRU'])
print('all_mae : ', _mae)
print('all_rmse : ', _rmse)
print('all_Rs : ', _Rs)
print('all_R : ', _R)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

save/geum/models/geum_pull/toc/
rnn_predict_day :  3
year : 2016 ~ 2018
run :  range(0, 12)
target :  toc   4
length :  26304
train=7, val=1, test=2
all_nse :  [0.61269263]
all_pbias :  [-2.79063775]
all_mae :  0.1791136794440638
all_rmse :  0.2280660136936422
all_Rs :  0.6712821442944846
all_R :  0.8193180971359565


In [112]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=train_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)

print('----------------------------')
print('train_nse : ', val_nse['GRU'])
print('train_pbias : ', val_pbias['GRU'])
print('train_mae : ', _mae)
print('train_rmse : ', _rmse)
print('train_Rs : ', _Rs)
print('train_R : ', _R)

#print('nse_train : ', val_nse['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
train_nse :  [0.61274409]
train_pbias :  [-4.7770261]
train_mae :  0.19330067819208888
train_rmse :  0.23779234193534024
train_Rs :  0.7615805891635141
train_R :  0.8726858479220997


In [113]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)

print('----------------------------')
print('val_nse : ', val_nse['GRU'])
print('val_pbias : ', val_pbias['GRU'])
print('val_mae : ', _mae)
print('val_rmse : ', _rmse)
print('val_Rs : ', _Rs)
print('val_R : ', _R)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
val_nse :  [-0.26147847]
val_pbias :  [5.54422616]
val_mae :  0.1972605432811614
val_rmse :  0.27877055203513235
val_Rs :  0.08502439253640223
val_R :  0.29158942459630155


In [114]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)

print('----------------------------')
print('test_nse : ', val_nse['GRU'])
print('test_pbias : ', val_pbias['GRU'])
print('test_mae : ', _mae)
print('test_rmse : ', _rmse)
print('test_Rs : ', _Rs)
print('test_R : ', _R)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
test_nse :  [0.54831109]
test_pbias :  [-1.25500084]
test_mae :  0.11404558613370859
test_rmse :  0.14112853106480708
test_Rs :  0.579981415348485
test_R :  0.7615651090671663


In [ ]:
#train_df.shape, val_df.shape, test_df.shape

In [ ]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvb111nfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

In [ ]:

sub_flag = True
history_nse = []

bob = 0
b = 1

while sub_flag:
    gru_model = model_gru(
        #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
        window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15, steps_per_epoch = 5,
        training_flag=True, checkpoint_path=model_path+"gru.ckpt", continue_train=False)
    
    #nse, _, _, _ = compa(
        #model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
        #predict_day = rnn_predict_day)
        #predict_day = 4)
    
    nse, _, _, _, _, _, _, _ = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
    
    if(nse > 0.5):
        sub_flag = False
   
    
    history_nse.append(nse)
    
    print('--------------------------')
    print(history_nse)
    print(nse)
    print('--------------------------')
    
        
    bob += b
    if(bob < -4):
        b = 1
    if(bob > 0):
        b = -1



In [ ]:

def compa2(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    

    
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    o_ = np.mean(label_day[:,predict_day,:])
    p_ = np.mean(pred_day[:,predict_day,:])
    
    oi = label_day[:,predict_day,:]
    pi = pred_day[:,predict_day,:]
    
    high = ((oi-o_)*(pi-p_)).sum()
    low = np.sqrt( ( (oi-o_)**2 ).sum() )
    low = low * np.sqrt( ( (pi-p_)**2 ).sum() )
    
    R = high/low
    RS = (R)**2
    
    return nse3, pbias3, inputs_day, pred_day, label_day, mae, rmse, RS, R

In [ ]:
__GAIN_TRAINING__, __RNN_TRAINING__

In [ ]:
#water_list = ['han', 'nak', 'geum', 'yeong']
#water_list = ['geum', 'yeong']
water_list = ['yeong']
for watererer in water_list:
    #file_parameters['watershed'] = 'geum'
    file_parameters['watershed'] = watererer
    parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
    with open(parameters_path, encoding='utf8') as json_file:
        parameters = json.load(json_file)

    data_parameters = parameters['data']
    interpolation_option = data_parameters['interpolation']
    colum_idx = data_parameters['columns']
    watershed = data_parameters['watershed']
    file_names = data_parameters['files']
    folder = data_parameters['directorys']
    for i in range(len(folder)):
        folder[i] = watershed+folder[i]

    if file_parameters['watershed'] == 'nak':
        run_num = range(len(folder))
    else:
        run_num = [0]

    real_df_all = pd.DataFrame([])
    target_all = target_mean = target_std = 0

    gain_val_performance = {}
    gain_performance = {}

    length = len(run_num)

    ddday = 31
    mmmonth = 12

    for i in range(length):

        idx = run_num[i]

        print('folder : ', data_path + folder[idx])

        df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                                   colum_idx[idx], interpolation=interpolation_option[idx],
                                  first_file_no=i, month=mmmonth, day=ddday)

        df_all, train_mean, train_std, df = normalize(df)


        if i == 0:
            dfff = df
            target_all = df_all
            target_std = train_std
            target_mean = train_mean
            start_year = str(times.iloc[0].year)
            end_year = str(times.iloc[-1].year)

        if interpolation_option[idx][0] == False:

            loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

            gain_calc_falg = True

            if __GAIN_TRAINING__ == True:
                gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
            else:
                for file in loadfiles:
                    if os.path.isfile('save/' + folder[idx]+file):
                        shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    else:
                        if file == 'miss.npy':
                            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')

            if gain_calc_falg == True:
                WindowGenerator.make_dataset = make_dataset_gain
                wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                              fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                              train_df = df_all, val_df = df_all, test_df = df_all, df = df)

                gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                                  training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

                gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
                gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

                if __GAIN_TRAINING__ == True:
                    if not os.path.exists('save/' + folder[idx]):
                        os.makedirs('save/'+folder[idx])
                    for file in loadfiles:
                        shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

                        ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

            else:
                gan = create_dataset_interpol(window=gain_in_setps, df=df)
        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)

        if i == 0 :
            real_df_all = pd.DataFrame(gan)
        else:
            real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)


    train_df, val_df, test_df = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
    teg_check = 'train=7, val=1, test=2'

    label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
    print("label_columns_indices:")
    print(label_columns_indices)
    target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
    target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
    out_features = [target_col_idx]
    out_num_features = len(out_features)

    print("target_col_idx : ", target_col_idx)
    print('out_num_features : ', out_num_features)

    target_list = ['do', 'toc', 'tn', 'tp', 'chl-a']
    #target_list = ['tp', 'chl-a']
    for rnn_target_column in target_list:
        #print(real_df_all.shape)
        #rnn_target_column = 'chl-a'
        #print(rnn_target_column)

        idx = [2, 4, 5, 6, 7]
        pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
        indices = {name: i for i, name in enumerate(idx)}
        target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
        label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
        print("label_columns_indices:")
        print(label_columns_indices)

        target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
        out_features = [target_col_idx]
        out_num_features = len(out_features)

        WindowGenerator.make_dataset = make_dataset_water
        multi_window = WindowGenerator(
            input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
            out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
            train_df=train_df, val_df=test_df, test_df=val_df)

        model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
        #model_path = "save/" + watershed + "models/" + pa[indices[target_col_idx]]
        print("save model path : ", model_path)

        if __RNN_TRAINING__:
            if not os.path.exists(model_path):
                os.makedirs(model_path)

        gru_model = model_gru(
            window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, 
            epochs=rnn_epochs, steps_per_epoch = rnn_steps_per_epoch,
            training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=rnn_continue_train)

        temp_df = val_df
        data_type = 'val'
        for data_type in range(0,4,1):
            if data_type == 0:
                temp_df = real_df_all
                data_type = 'all'
            elif data_type == 1:
                temp_df = train_df
                data_type = 'train'
            elif data_type == 2:
                temp_df = val_df
                data_type = 'val'
            elif data_type == 3:
                temp_df = test_df
                data_type = 'test'

            for rnn_predict_day in range(2,5,1):
                _nse, _pbias, _input_day, _pred_day, _label_day, _mae, _rmse, _Rs, _R = compa2(
                    model=gru_model,df=temp_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
                    predict_day = rnn_predict_day)

                print(model_path)
                print("year : " + start_year + " ~ "+ end_year)
                print('run : ', run_num)
                print('target : ', rnn_target_column)
                print(teg_check)

                image_file_name = file_parameters['watershed']+'_'+rnn_target_column+'_'+data_type+'_'+str(rnn_predict_day+1)+'.png'
                print(image_file_name)

                outdata = {
                    'image_file_name' : image_file_name,
                    'watershed':file_parameters['watershed'],
                    'target':rnn_target_column,
                    'type':data_type,
                    'pred_day':rnn_predict_day+1,
                    'year':start_year + " ~ "+ end_year,
                    'nse':_nse, 
                    'pbias':_pbias,
                    'mae':_mae,
                    'rmse':_rmse,
                    'rs':_Rs,
                    'r':_R,
                }
                outdata = pd.DataFrame(outdata)
                outdata.to_csv('temp/data.csv', mode='a', header=False)

                plt.figure(figsize=(12,12),dpi=96)
                _input_index = np.array(range(_input_day.shape[0]))
                _label_index = _input_index + (7 + rnn_predict_day)
                plt.plot(_input_index, _input_day[:, 0, :], label='input')
                plt.plot(_label_index, _label_day[:, rnn_predict_day, :], label='label')
                plt.plot(_label_index, _pred_day[:, rnn_predict_day, :], label='pred')
                plt.legend()
                plt.savefig('temp/'+image_file_name)

    print('end')

In [ ]:
#water_list = ['han', 'nak', 'geum', 'yeong']
#water_list = ['geum', 'yeong']

for ddddd in [1]:
    label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
    print("label_columns_indices:")
    print(label_columns_indices)
    target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
    target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
    out_features = [target_col_idx]
    out_num_features = len(out_features)

    print("target_col_idx : ", target_col_idx)
    print('out_num_features : ', out_num_features)

    target_list = ['do', 'toc', 'tn', 'tp', 'chl-a']
    #target_list = ['tp', 'chl-a']
    for rnn_target_column in target_list:
        #print(real_df_all.shape)
        #rnn_target_column = 'chl-a'
        #print(rnn_target_column)

        idx = [2, 4, 5, 6, 7]
        pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
        indices = {name: i for i, name in enumerate(idx)}
        target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
        label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
        print("label_columns_indices:")
        print(label_columns_indices)

        target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
        out_features = [target_col_idx]
        out_num_features = len(out_features)

        WindowGenerator.make_dataset = make_dataset_water
        multi_window = WindowGenerator(
            input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
            out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
            train_df=train_df, val_df=test_df, test_df=val_df)

        model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
        #model_path = "save/" + watershed + "models/" + pa[indices[target_col_idx]]
        print("save model path : ", model_path)

        if __RNN_TRAINING__:
            if not os.path.exists(model_path):
                os.makedirs(model_path)

        gru_model = model_gru(
            window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, 
            epochs=rnn_epochs, steps_per_epoch = rnn_steps_per_epoch,
            training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=rnn_continue_train)

        temp_df = val_df
        data_type = 'val'
        for data_type in range(0,4,1):
            if data_type == 0:
                temp_df = real_df_all
                data_type = 'all'
            elif data_type == 1:
                temp_df = train_df
                data_type = 'train'
            elif data_type == 2:
                temp_df = val_df
                data_type = 'val'
            elif data_type == 3:
                temp_df = test_df
                data_type = 'test'

            for rnn_predict_day in range(2,5,1):
                _nse, _pbias, _input_day, _pred_day, _label_day, _mae, _rmse, _Rs, _R = compa2(
                    model=gru_model,df=temp_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
                    predict_day = rnn_predict_day)

                print(model_path)
                print("year : " + start_year + " ~ "+ end_year)
                print('run : ', run_num)
                print('target : ', rnn_target_column)
                print(teg_check)

                image_file_name = file_parameters['watershed']+'_'+rnn_target_column+'_'+data_type+'_'+str(rnn_predict_day+1)+'.png'
                print(image_file_name)

                outdata = {
                    'image_file_name' : image_file_name,
                    'watershed':file_parameters['watershed'],
                    'target':rnn_target_column,
                    'type':data_type,
                    'pred_day':rnn_predict_day+1,
                    'year':start_year + " ~ "+ end_year,
                    'nse':_nse, 
                    'pbias':_pbias,
                    'mae':_mae,
                    'rmse':_rmse,
                    'rs':_Rs,
                    'r':_R,
                }
                outdata = pd.DataFrame(outdata)
                outdata.to_csv('temp/data.csv', mode='a', header=False)

                plt.figure(figsize=(12,12),dpi=96)
                _input_index = np.array(range(_input_day.shape[0]))
                _label_index = _input_index + (7 + rnn_predict_day)
                plt.plot(_input_index, _input_day[:, 0, :], label='input')
                plt.plot(_label_index, _label_day[:, rnn_predict_day, :], label='label')
                plt.plot(_label_index, _pred_day[:, rnn_predict_day, :], label='pred')
                plt.legend()
                plt.savefig('temp/'+image_file_name)

    print('end')